In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True) #do not miss this line
import warnings

warnings.filterwarnings('ignore')

In [48]:
data = pd.read_csv('data_apparment',sep = ',', index_col= 'Unnamed: 0')
data.head(3)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
19,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.580503566,-58.4058744847",-34.580504,-58.405874,...,96.0,3365.384615,3645.833333,NaN,3.0,NaN,http://www.properati.com.ar/15bq8_venta_depart...,Excelente semipiso al contra frente en Bulnes ...,"Bulnes y Libertador: espectacular pulmón, con ...",https://thumbs4.properati.com/8/1y9fKHLee-aQQj...


In [129]:
#busqueda de amenities en dataset final

#match amenities
pattern_amenities = "lavadero|Lavadero|LAVADERO|lavanderia|lavandería|Lavanderia|Lavandería|LAVANDERIA|LAVANDERÍA|laundry|LAUNDRY|Laundry" 
match_pattern_amenities = data.description.str.contains(pattern_amenities, regex= True)
print('Coincidencias amenities ' + str(match_pattern_amenities.sum()))
mask_match_pattern_amenities_notnull = match_pattern_amenities.notnull()

###"solarium|SOLARIUM|Solarium|Terraza|terraza|TERRAZA|gimnasio|GIMNASIO|Gimnasio|GYM|gym|Gym|Cochera|cochera|COCHERA|GARAGE|Garage|garage|parrilla|Parrilla|PARRILLA|pileta|Pileta|PILETA|Piscina|piscina|PISCINA|lavadero|Lavadero|LAVADERO|lavanderia|lavandería|Lavanderia|Lavandería|LAVANDERIA|LAVANDERÍA|PATIO|JARDIN|JARDÍN|Patio|patio|Jardin|Jardín|jardin|jardín|balcon|balcón|BALCON|BALCÓN|Balcón|Balcon|sum|SUM|Sum" 

Coincidencias amenities 2900


In [130]:
#incorporación de columna 'amenities' al data final
data.loc[mask_match_pattern_amenities_notnull, "lavadero"] = match_pattern_amenities[mask_match_pattern_amenities_notnull].astype(int)

In [237]:
data.avenida.value_counts()/data.shape[0]

1    0.643162
0    0.356838
Name: avenida, dtype: float64

In [324]:
data.shape

(6325, 36)

In [325]:
data.to_csv('data_regex')

In [315]:
columnas_a_seleccionar = ['place_name','price_usd_per_m2','surface_total_in_m2','lat','lon','floor','rooms','description','title','avenida','amenities','balcon','gimnasio','SUM','cochera','parrilla','pileta','patio','terraza','lavadero']
data_seleccion = data[columnas_a_seleccionar]

In [326]:
data_seleccion.head(3)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,gimnasio,SUM,cochera,parrilla,pileta,patio,terraza,amenities,avenida,lavadero
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,0,0,0,0,1,0,1,1,1,1
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,0,0,0,0,1,0,1,1,1,1
19,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.580503566,-58.4058744847",-34.580504,-58.405874,...,0,0,0,0,0,0,1,1,0,0


In [317]:
place_dummy = pd.get_dummies(data_seleccion['place_name'],drop_first=True)

In [318]:
data_seleccion = pd.concat([data_seleccion,place_dummy],axis=1)

In [319]:
data_seleccion.head(3)

,place_name,price_usd_per_m2,surface_total_in_m2,lat,lon,floor,rooms,description,title,avenida,...,parrilla,pileta,patio,terraza,lavadero,Belgrano,Nuñez,Palermo,Palermo Hollywood,Recoleta
7,Belgrano,3066.666667,45.0,-34.559873,-58.443362,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,1,...,0,1,0,1,1,1,0,0,0,0
8,Belgrano,3000.000000,65.0,-34.559873,-58.443362,NaN,NaN,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",1,...,0,1,0,1,1,1,0,0,0,0
19,Palermo,3365.384615,104.0,-34.580504,-58.405874,NaN,3.0,Excelente semipiso al contra frente en Bulnes ...,"Bulnes y Libertador: espectacular pulmón, con ...",0,...,0,0,0,1,0,0,0,1,0,0


In [320]:
data_seleccion.lat.isnull().sum()

1637

In [321]:
data_seleccion.dropna(axis=0,subset=['lat'],inplace=True)

In [308]:
##data_seleccion.dropna(axis=0,subset=['rooms'],inplace=True)

In [322]:
data_seleccion.shape

(4688, 25)

In [323]:
data_seleccion.to_csv('data_seleccion')

In [150]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [306]:
# Definimos una función que acepta una lista de features, hace el split entre train y test,
# reservando un 25% de las observaciones para testeo, y devuelve la prueba RMSE.

def train_test_rmse(feature_cols):
    X = data_seleccion[feature_cols]
    y = data_seleccion.price_usd_per_m2
    # Como estamos trabajando con observaciones ordenadas en el tiempo, ponemos
    # shuffle=False para evitar data leakage
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    return  print('MAE:', metrics.mean_absolute_error(y_test, y_pred),
    print('MSE:', metrics.mean_squared_error(y_test, y_pred),
    print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)),
    print('R2:', metrics.r2_score(y_test, y_pred)))))

In [295]:
train_test_rmse(['surface_total_in_m2','rooms'])

R2: -0.02458053611264388
RMSE: 812.8030398013859 None
MSE: 660648.7815103733 None
MAE: 620.2012203400683 None


In [275]:
train_test_rmse(['surface_total_in_m2','lat','lon','avenida','balcon','gimnasio','SUM','cochera','parrilla','pileta','patio','terraza','lavadero','Belgrano','Palermo','Nuñez','Palermo Hollywood','Recoleta'])

R2: 0.2756698280178016
RMSE: 670.0711790936505 None
MSE: 448995.38505195506 None
MAE: 502.1032462924897 None


In [242]:
data_seleccion['surface_total_in_m2_2'] = data['surface_total_in_m2']**2
data_seleccion.head()

,place_name,price_usd_per_m2,surface_total_in_m2,floor,rooms,description,title,avenida,amenities,balcon,...,pileta,patio,terraza,lavadero,Belgrano,Nuñez,Palermo,Palermo Hollywood,Recoleta,surface_total_in_m2_2
7,Belgrano,3066.666667,45.0,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,1,1,0,...,1,0,1,1,1,0,0,0,0,2025.0
8,Belgrano,3000.000000,65.0,NaN,NaN,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",1,1,0,...,1,0,1,1,1,0,0,0,0,4225.0
19,Palermo,3365.384615,104.0,NaN,3.0,Excelente semipiso al contra frente en Bulnes ...,"Bulnes y Libertador: espectacular pulmón, con ...",0,1,1,...,0,0,1,0,0,0,1,0,0,10816.0
21,Palermo,2292.372881,118.0,NaN,4.0,"EXCELENTE ZONA, MULTIPLES MEDIOS DE TRANSPORTE...",Departamento de 4 ambientes en Venta en Palermo,1,1,0,...,0,1,0,0,0,0,1,0,0,13924.0
60,Nuñez,3460.000000,50.0,NaN,2.0,ESPECTACULAR DEPARTAMENTO DE 2 AMBIENTES CON C...,"VENTA 2 AMB CON COCH, PISCINA PARA INVERSION...",1,1,1,...,1,0,1,0,0,1,0,0,0,2500.0


In [239]:
import statsmodels.api as sm

In [310]:
feature_cols = ['surface_total_in_m2','lat','lon','rooms','avenida','balcon','gimnasio','SUM','cochera','parrilla','pileta','patio','terraza','lavadero','Belgrano','Palermo','Nuñez','Palermo Hollywood','Recoleta']

X = data_seleccion[feature_cols]

y = data_seleccion.price_usd_per_m2

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
print(model.summary())

##

                            OLS Regression Results                            
Dep. Variable:       price_usd_per_m2   R-squared:                       0.335
Model:                            OLS   Adj. R-squared:                  0.330
Method:                 Least Squares   F-statistic:                     72.80
Date:                Fri, 13 Jan 2023   Prob (F-statistic):          1.77e-226
Time:                        21:05:35   Log-Likelihood:                -21826.
No. Observations:                2771   AIC:                         4.369e+04
Df Residuals:                    2751   BIC:                         4.381e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 1.52e+06   1